# 한국어 필기체 인식기
### Hand-written Korean Language Recognizer

### 1. 데이터셋 : phd08
###

In [1]:
#-*- coding: utf-8 -*-

import os
import numpy as np
import cv2

Label 종류 가져오기

In [2]:
dataset_path = '/Users/kimsu/datasets/korean_image/phd08/'
txt_files = os.listdir(os.path.join(dataset_path, 'txt'))
txt_files.sort()

data_labels = [ch.split('.')[0] for ch in txt_files if ch.split('.')[-1] == 'txt']
del txt_files

with open(os.path.join(dataset_path, 'labels_to_index.txt'), 'r') as f:
    labels = [line.strip() for line in f]
    
label_to_idx = {l:i for i, l in enumerate(labels)}
idx_to_label = {i:l for i, l in enumerate(labels)}

In [3]:
label_to_idx[idx_to_label[2]]

2

PHD08 데이터셋 txt 파일 --> png 이미지로 변환

In [24]:
def phd08_txt_to_image(txt_path, label_name, img_path, label_to_idx):
    with open(os.path.join(txt_path, label_name + '.txt'), 'r') as file:
        data_checker = 0
        for line in file.readlines():
            if not line.strip():
                # End of line
                font_array = cv2.GaussianBlur(font_array, (7, 7), 0.5)
                if not os.path.exists(img_path):
                    os.makedirs(img_path)
                font_array *= 255
                font_array = font_array.astype(np.uint8)
                cv2.imwrite(os.path.join(img_path, save_file_name), font_array)
                
                data_checker = 0
            else:
                if data_checker == 0:
                    save_file_name = '%06d_%s.png' % (label_to_idx[label_name], line.strip())
                elif data_checker == 1:
                    height, width = line.strip().split(' ')
                    height = int(height)
                    width = int(width)
                    font_array = np.zeros([height, width])
                else:
                    font_array[data_checker-2] = list(map(int, line.strip()))
                data_checker += 1

In [26]:
dataset_path = '/Users/kimsu/datasets/korean_image/phd08'
phd08_txt_to_image(txt_path=dataset_path + '/txt/',
                  label_name = labels[0],
                  img_path=dataset_path + '/png',
                  label_to_idx=label_to_idx)

자모 분해

In [29]:
import re

In [55]:
BASE_CODE, INIT_CODE, MED_CODE = 44032, 588, 28

init_chars_list=['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ',
                 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ']
med_chars_list=['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ',
                'ㅣ', 'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅝ',
                'ㅞ', 'ㅟ', 'ㅢ']
final_chars_list=['', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ',
                  'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㅆ',
                  'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']

In [31]:
init_chars_list=['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ',
                 'ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
med_chars_list=['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ',
                'ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
final_chars_list=['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ',
                  'ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ',
                  'ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']

In [31]:
def list_to_string(src):
    string = ""
    for arg in src:
        string += arg
    return string

def load_korean_chars(path):
    init_list = []
    med_list = []
    final_list = []
    with open(path, 'r') as f:
        init_string = f.readline().strip()
        med_string = f.readline().strip()
        final_string = f.readline().strip()
        
        init_list = [ch for ch in init_string]
        med_list = [ch for ch in med_string]
        final_list = [""] + [ch for ch in final_string]
        
    return init_list, med_list, final_list

def save_korean_chars(path, init_list, med_list, final_list):
    with open(path, 'w') as f:
        init_string = list_to_string(init_list)
        med_string = list_to_string(med_list)
        final_string = list_to_string(final_list)
        
        f.write("%s\n" % init_string)
        f.write("%s\n" % med_string)
        f.write("%s\n" % final_string)

In [32]:
init_list, med_list, final_list = load_korean_chars('/Users/kimsu/src/koreanRecognition/ch_list_lut.txt')

In [37]:
init_ch_to_idx = {c:i for i, c in enumerate(init_list)}
med_ch_to_idx = {c:i for i, c in enumerate(med_list)}
final_ch_to_idx = {c:i for i, c in enumerate(final_list)}

init_idx_to_ch = {i:c for i, c in enumerate(init_list)}
med_idx_to_ch = {i:c for i, c in enumerate(med_list)}
final_idx_to_ch = {i:c for i, c in enumerate(final_list)}

In [125]:
def decompose_korean_letter(ch):
    args = []
    args_idx = []
    if len(ch) > 1:
        print(1)
        c1, c2, c3 = ch[0], ch[1], ch[2] if len(ch) == 3 else ''
        args = [c1, c2, c3]
        args_idx = [init_ch_to_idx[c1], med_ch_to_idx[c2], final_ch_to_idx[c3]]
    elif re.match('.*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*', ch) is not None:
        char_code = ord(ch) - BASE_CODE
        c1 = int(char_code / INIT_CODE)
        c2 = int((char_code - (INIT_CODE*c1)) / MED_CODE)
        c3 = int((char_code - (INIT_CODE*c1)) - (MED_CODE*c2))
        
        args = [init_idx_to_ch[c1], med_idx_to_ch[c2], final_idx_to_ch[c3]]
        args_idx = [c1, c2, c3]
    else:
        args = [ch]
        args_idx = [-1]
    
    return args, args_idx

def decompose_korean_letters(chs):
    args_list = []
    args_idx_list = []
    for ch in chs:
        args, args_idx = decompose_korean_letter(ch)
        args_list.append(args)
        args_idx_list.append(args_idx)
    
    return args_list, args_idx_list

def assemble_korean_letter(idxs):
    return BASE_CODE + idxs[0] * INIT_CODE + idxs[1] * MED_CODE + idxs[2]

In [126]:
letters, idxs = decompose_korean_letter('뽀')
letters, idxs

(['ㅃ', 'ㅗ', ''], [8, 8, 0])

In [129]:
decompose_korean_letters('나비야 hi')

([['ㄴ', 'ㅏ', ''], ['ㅂ', 'ㅣ', ''], ['ㅇ', 'ㅑ', ''], [' '], ['h'], ['i']],
 [[2, 0, 0], [7, 20, 0], [11, 2, 0], [-1], [-1], [-1]])

In [70]:
code = assemble_korean_letter(idxs)